# Variational Autoencoder (VAE) on MNIST — Beginner Friendly Walkthrough
This notebook explains a Variational Autoencoder (VAE) trained on the MNIST dataset using PyTorch.

Each step is annotated with detailed comments to help beginners understand what's happening.

In [ ]:
!pip install torch torchvision

## 1. Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


## 2. Load and Prepare the MNIST Dataset

In [ ]:
# We transform MNIST images into tensors.
transform = transforms.ToTensor()

# Download and load the training data
train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# DataLoader for batching and shuffling
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)


## 3. Define the VAE Model

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim=784, hidden_dim=400, latent_dim=20):
        super(VAE, self).__init__()
        # Encoder layers: input -> hidden -> (mu, logvar)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)      # outputs mean of q(z|x)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)  # outputs log-variance of q(z|x)

        # Decoder layers: latent -> hidden -> reconstruction
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        # Apply a hidden layer then split into mean and logvar
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        # Apply the reparameterization trick
        std = torch.exp(0.5 * logvar)      # standard deviation
        eps = torch.randn_like(std)        # random normal noise
        return mu + eps * std              # sample z

    def decode(self, z):
        # Reconstruct input from latent representation
        h = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h))  # Output in [0, 1] range for binary MNIST

    def forward(self, x):
        # Full VAE forward pass
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar


## 4. Define the ELBO Loss

In [ ]:
def elbo_loss(recon_x, x, mu, logvar):
    # Binary cross-entropy for reconstruction
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # KL divergence term to regularize q(z|x) against standard normal p(z)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Total loss is negative ELBO
    return BCE + KLD


## 5. Train the VAE

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, _ in train_loader:
        x = x.view(-1, 784).to(device)               # Flatten 28x28 images into 784 vectors
        recon_x, mu, logvar = model(x)               # Forward pass
        loss = elbo_loss(recon_x, x, mu, logvar)     # Compute loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader.dataset):.2f}")


## 6. Visualize Original and Reconstructed Digits

In [ ]:
model.eval()
with torch.no_grad():
    x, _ = next(iter(train_loader))
    x = x.view(-1, 784).to(device)
    recon_x, _, _ = model(x)

    # Convert back to image format
    x = x.view(-1, 1, 28, 28).cpu()
    recon_x = recon_x.view(-1, 1, 28, 28).cpu()

    fig, axs = plt.subplots(2, 10, figsize=(15, 3))
    for i in range(10):
        axs[0, i].imshow(x[i][0], cmap='gray')
        axs[0, i].axis('off')
        axs[1, i].imshow(recon_x[i][0], cmap='gray')
        axs[1, i].axis('off')
    axs[0, 0].set_ylabel("Original", fontsize=12)
    axs[1, 0].set_ylabel("Reconstruction", fontsize=12)
    plt.tight_layout()
    plt.show()
